In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns

from qolmat.imputations.imputers import ImputerMICE, ImputerRegressor
from qolmat.imputations import estimators
from qolmat.benchmark.metrics import get_metric
from qolmat.benchmark import missing_patterns
from qolmat.benchmark import comparator
from qolmat.utils import plot
from qolmat.imputations import imputers

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.base import (
    BaseEstimator,
    ClassifierMixin,
    RegressorMixin,
)
from sklearn.compose import make_column_selector as selector

from category_encoders.one_hot import OneHotEncoder

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
df = pd.read_csv("../data/titanic.csv", sep=";")

In [4]:
y = df["survived"] == 1

In [5]:
df = df.dropna(how="all")
df = df.drop(columns=["pclass", "survived", "name", "home.dest", "cabin", "ticket", "boat", "body"])

In [6]:
df["age"] = pd.to_numeric(df["age"], errors="coerce")
df["fare"] = pd.to_numeric(df["fare"].str.replace(",", ""), errors="coerce")

In [7]:
df.head(3)

,sex,age,sibsp,parch,fare,embarked
0,female,29.0,0.0,0.0,2113375.0,S
1,male,NaN,1.0,2.0,1515500.0,S
2,female,2.0,1.0,2.0,1515500.0,S


In [8]:
selector_cat = selector(dtype_exclude=np.number)
cols_cat = selector_cat(df)

In [9]:
imputer_rpca = imputers.ImputerRpcaNoisy()
ohe = OneHotEncoder(handle_unknown="ignore", handle_missing="return_nan", use_cat_names=True, cols=cols_cat)
bt = estimators.BinTransformer()
wrapper = Pipeline(steps=[("OneHotEncoder", ohe), ("BinTransformer", bt)])

imputer_rpca = estimators.WrapperTransformer(imputer_rpca, wrapper)

In [10]:
pipestimator = estimators.make_robust_MixteHGB(allow_new=False)
imputer_hgb = ImputerRegressor(estimator=pipestimator, handler_nan="none")

In [11]:
imputer_simple = imputers.ImputerSimple()

In [12]:
dict_imputers = {"Simple": imputer_simple, "HGB": imputer_hgb, "RPCA": imputer_rpca}
cols_to_impute = df.columns
ratio_masked = .1
generator_holes = missing_patterns.UniformHoleGenerator(n_splits=2, subset=cols_to_impute, ratio_masked=ratio_masked, sample_proportional=False)
# metrics = ["mae", "wmape", "KL_columnwise", "frechet"]
metrics = ["mae"]

In [13]:
comparison = comparator.Comparator(
    dict_imputers,
    cols_to_impute,
    generator_holes = generator_holes,
    metrics=metrics,
    max_evals=2,
)
results = comparison.compare(df)
results.style.highlight_min(color="lightgreen", axis=1)

Tested model: ImputerSimple
Tested model: ImputerRegressor
Tested model: WrapperTransformer
